In [1]:
kaggle=False
kaggle_path='/kaggle/input/nlp-getting-started/train.csv'
local_path='train.csv'
import os
if kaggle:
    os.system('pip install neptune')
    
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import pandas as pd 
import numpy as np
from tqdm.notebook import tqdm
# models: https://github.com/huggingface/transformers/tree/main/examples/research_projects/distillation

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')


df=pd.read_csv(kaggle_path if kaggle else local_path)


def prepare_df(df):
    df=df.copy()
    df.keyword=df.keyword.str.replace('%20',' ')
    
    # enrich text with location and text, when they are not null
    df['text']='location: ' + df['location'].fillna('') + ' | keyword: ' +  df['keyword'].fillna('') + ' | text: ' + df['text'].fillna('')
    # drop location and keyword
    df=df.drop(columns=['location','keyword'])
    # drop id
    df=df.drop(columns=['id'])
    return df

df=prepare_df(df)
df.sample(10)



,text,target
4774,location: Reddit | keyword: lightning | text:...,1
6441,location: Chicago IL | keyword: suicide bombin...,1
2656,"location: Brasil,SP | keyword: detonate | text...",0
6063,location: Alger-New York-San Francisco | keywo...,1
2599,"location: North Port, FL | keyword: destroyed ...",0
1594,"location: Madrid, Comunidad de Madrid | keywor...",0
1544,"location: Tyler, TX | keyword: chemical emerge...",0
5046,location: | keyword: mudslide | text: @nikist...,0
6245,"location: Brooklyn, NY | keyword: snowstorm | ...",0
52,location: Barbados | keyword: ablaze | text: S...,0


In [2]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

In [3]:
# define dataset and dataloader

SEQ_LEN = 50

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, test=False):
        self.df = df
        self.test = test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text=self.df.iloc[idx]['text']

        input_ids = tokenizer.encode(text, add_special_tokens=True)
        attention_mask = [1] * len(input_ids)

        if (len(input_ids)<SEQ_LEN):
            input_ids=input_ids+[0]*(SEQ_LEN-len(input_ids))
            attention_mask=attention_mask+[0]*(SEQ_LEN-len(attention_mask))
        elif (len(input_ids)>SEQ_LEN):
            input_ids=input_ids[:SEQ_LEN]
            attention_mask=attention_mask[:SEQ_LEN]

        if self.test:
            return torch.tensor(input_ids), torch.tensor(attention_mask)
        else:
            label=self.df.iloc[idx]['target']
            return torch.tensor(input_ids), torch.tensor(label), torch.tensor(attention_mask)
        
train_dataset = Dataset(train_df)
valid_dataset = Dataset(valid_df)






In [4]:

class ClassifierModel(torch.nn.Module):
    def __init__(self, p_dropout=0.5):
        super().__init__()
        #self.model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

        self.model = DistilBertModel.from_pretrained('distilbert-base-cased')

        #self.freeze()
        self.linear1 = torch.nn.Linear(768, 2)
        #self.linear2=torch.nn.Linear(350, 2)
        #self.maxpool=torch.nn.MaxPool1d(SEQ_LEN)
        self.dropout=torch.nn.Dropout(p_dropout)
        #self.batchnorm=torch.nn.BatchNorm1d(350)
        #self.relu=torch.nn.ReLU()
        

    def freeze(self):
        for param in self.model.parameters():
            param.requires_grad = False

    def forward(self, x, attention_mask=None):
        output = self.model(x, attention_mask=attention_mask)["last_hidden_state"]
        pooled_output = torch.mean(output, dim=1)
        pooled_output = self.dropout(pooled_output)
        output = self.linear1(pooled_output)
        #output = self.relu(output)
        #output = self.dropout(output)
        #output = self.batchnorm(output)
        #output = self.linear2(output)
        return output
    
my_model=ClassifierModel(p_dropout=0.5)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
for xb, yb, att_mask in train_dataloader:
    print(xb.shape)
    print(yb.shape)
    print(att_mask.shape)
    print(my_model(xb, attention_mask=att_mask).shape)
    break



Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([32, 50])
torch.Size([32])
torch.Size([32, 50])
torch.Size([32, 2])


In [5]:

import neptune

class Learner():
    def __init__(self, model, train_dataloader, valid_dataloader, batch_size=32):
        self.model=model
        self.loss_fn=torch.nn.CrossEntropyLoss()
        self.device=torch.device("cpu")
        if torch.cuda.is_available():
            self.device=torch.device("cuda")
        #elif torch.backends.mps.is_available():
        #    self.device=torch.device("mps")

        self.model.to(self.device)
        self.run=neptune.init_run(
            project="bernd.heidemann/clickbait-classification",
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzNjBlYzVkNi0zZTUwLTQ1ODYtODhlNC02NDUxNDg0MDdjNzUifQ==",
        )
        self.batch_size=batch_size
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader

    def fit(self, lr=0.001, epochs=10):
        self.run["parameters"] = {
            "lr": lr,
            "epochs": epochs,
            "batch_size": self.batch_size,
        }
        optimizer=torch.optim.AdamW(self.model.parameters(), lr=lr)
        scheduler=scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        bar = tqdm(total=len(self.train_dataloader) * epochs, desc="Training")
        bar.set_description("Epoch 0/{}".format(epochs))

        for epoch in range(epochs):
            self.model.train()            
            for xb, yb, att_mask in self.train_dataloader:
                
                xb=xb.to(self.device)
                yb=yb.to(self.device)
                att_mask=att_mask.to(self.device)
                pred=self.model(xb, attention_mask=att_mask)
                loss=self.loss_fn(pred, yb)
                self.run["train_loss"].log(loss.item())
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                bar.update(1)
            scheduler.step()
            self.model.eval()
            # log current state to neptune
            if self.valid_dataloader is not None:
                metrics=self.get_accuracy()
                self.run["valid_accuracy"].log(metrics["accuracy"])
                self.run["valid_loss"].log(metrics["loss"])
                bar.set_description("Epoch {}/{} validAccuracy: {:.2f} validLoss: {:.2f}".format(epoch+1, epochs, metrics["accuracy"], metrics["loss"]))
            
                
    def get_accuracy(self):
        self.model.eval()
        with torch.no_grad():
            correct=0
            losses=[]
            for xb, yb, att_mask in self.valid_dataloader:
                xb=xb.to(self.device)
                yb=yb.to(self.device)
                att_mask=att_mask.to(self.device)
                pred=self.model(xb, attention_mask=att_mask)
                loss=self.loss_fn(pred, yb)
                losses.append(loss.item())
                pred=torch.argmax(pred, dim=1)
                correct+=torch.sum(pred==yb).item()
            return {
                "accuracy": correct/len(valid_dataset),
                "loss": np.mean(losses)
            }
    

In [6]:

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)

learner=Learner(my_model, train_dataloader, valid_dataloader, batch_size=128)
learner.fit(lr=0.0001, epochs=2)

/var/folders/yl/qjs6b9wn4zx7nh630c4my9lw0000gn/T/ipykernel_72105/1806302871.py:14: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  self.run=neptune.init_run(


https://app.neptune.ai/bernd.heidemann/clickbait-classification/e/CLIC-104


Training:   0%|          | 0/382 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
learner.get_accuracy()

{'accuracy': 0.8332239001969797, 'loss': 0.4055103212594986}

In [ ]:
full_dataset = Dataset(df, test=True)
full_dataloader = torch.utils.data.DataLoader(full_dataset, batch_size=32, shuffle=False)

learner=Learner(my_model, full_dataloader, None, batch_size=128)
learner.fit(lr=0.0001, epochs=2)

In [ ]:
local_test_path='test.csv'
kaggle_test_path='/kaggle/input/nlp-getting-started/test.csv'

df_submission_test_data=pd.read_csv(kaggle_test_path if kaggle else local_test_path)
df_submission_test_data.sample(5)

,id,keyword,location,text
3095,10242,volcano,"Baker, Louisiana",what's the hottest thing you've ever seen..fir...
1588,5369,fire%20truck,Los Angeles,#Irvine #SigAlert update: 133 S now closed at ...
484,1571,bomb,NaN,@MattieLBreaux u the bomb ????????????????
2197,7351,obliterate,NaN,@RondaRousey Though I am certain that you woul...
196,632,arsonist,"San Luis Obispo, California",Suspected serial arsonist arrested in NorCal h...


In [ ]:
test_dataset=Dataset(df_submission_test_data, test=True)
test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

for xb, att_mask in test_loader:
    xb=xb.to(learner.device)
    att_mask=att_mask.to(learner.device)
    print(xb.shape)
    print(att_mask.shape)
    print(my_model(xb, attention_mask=att_mask).shape)
    break


torch.Size([32, 30])
torch.Size([32, 30])
torch.Size([32, 2])


In [ ]:
predictions=[]

for xb, att_mask in test_loader:
    xb=xb.to(learner.device)
    att_mask=att_mask.to(learner.device)
    pred=my_model(xb, attention_mask=att_mask)
    pred=torch.argmax(pred, dim=1)
    predictions+=pred.tolist()

df_submission_test_data['target']=predictions
df_submission_test_data[['id', 'target']].to_csv('submission.csv', index=False)

In [ ]:
# 0.80692